# Tiger Tutorial: Solving POMDPs

This tutorial outlines how to define a POMDP using the [POMDPs.jl](https://github.com/sisl/POMDPs.jl) interface. We will go through a simple problem simply known as the tiger problem (we will refer to it as the tiger POMDP). After defining the tiger POMDP, we will use QMDP and SARSOP to solve the POMDP. If you are new to working with this package, check out the [tutorial](http://nbviewer.ipython.org/github/sisl/POMDPs.jl/blob/master/examples/GridWorld.ipynb) on MDPs first.

## Problem Overview
In the tiger POMDP, the agent is tasked with escaping from a room. There are two doors leading out of the room. Behind one of the doors is a tiger, and behind the other is sweet, sweet freedom. If the agent opens the door and finds the tiger, it gets eaten (and receives a reward of -100). If the agent opens the other door, it escapes and receives a reward of 10. The agent can also listen. Listening gives a noisy measuremnt of which door the tiger is hiding behind. Listening gives the agent the correct location of the tiger 85% of the time. The agent receives a reward of -1 for listening. 

In [1]:
# first import POMDPs.jl
using POMDPs

## POMDP 
A POMDP is defined by the tuple
$$(\mathcal{S}, \mathcal{A}, \mathcal{Z}, T, R, O).$$
In addition to the familiar, state $\mathcal{S}$ and action $\mathcal{A}$ spaces, we must also define an observation space $\mathcal{Z}$ and an observation function $O$. The POMDP problem definition may be similar to the one for MDP. For example, if you wanted to add state uncertaitniy to your problem, you can define the observation space, and observation function in addition to your previous MDP definition.

Before defining the spaces for this problem, let's first deinfe the concrete type for the tiger POMDP. 

In [2]:
type TigerPOMDP <: POMDP
    r_listen::Float64 # reward for listening (default -1)
    r_findtiger::Float64 # reward for finding the tiger (default -100)
    r_escapetiger::Float64 # reward for escaping (default 10)
    p_listen_correctly::Float64 # prob of correctly listening (default 0.85)
    discount_factor::Float64 # discount
end
# default constructor
function TigerPOMDP()
    return TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95)
end

TigerPOMDP (constructor with 3 methods)

There are a number of parameters in the problem definition, but we can treat them all as constants. You can read more about the Tiger problem and POMDPs [here](http://www.techfak.uni-bielefeld.de/~skopp/Lehre/STdKI_SS10/POMDP_tutorial.pdf#page=28). However, we created a default constructor that allows us to initialize the tiger POMDP by simply running:

In [3]:
pomdp = TigerPOMDP()

TigerPOMDP(-1.0,-100.0,10.0,0.85,0.95)

## States
We define a concrete type to represent the state of the tiger POMDP. The only thing we need to know to represent the problem is which door the tiger is hiding behind. 

In [4]:
type TigerState <: State
    tigerleft::Bool
end

Since the state is a binary value, we represent it as a boolean, but we could have represented it as an integer or any other sensible type.

## Actions
There are three possible actions our agent can take: open the left door, open the right door, and listen. For clarity, we will represent these with symbols.

In [5]:
type TigerAction <: Action
    act::Symbol
end

We will represent our actions with the following symbols: open left (:openl), open right (:openr), and listen (:listen). For example, the action below represnts listening:

In [6]:
a = TigerAction(:listen)

TigerAction(:listen)

## Observations
There are two possible observations: the agent either hears the tiger behind the left door or behind the right door. We use a boolean to represent the observations:

In [7]:
type TigerObservation <: Observation
    obsleft::Bool 
end

## Spaces
Let's define our state, action and observation spaces.

### State Space
There are only two states in the tiger POMDP: the tiger is either behind the left door or behind the right door. Our state space is simply an array of the states in the problem.

In [8]:
type TigerStateSpace <: AbstractSpace
    states::Vector{TigerState} 
end

Let's now define the ```states``` and the ```domain``` functions. Recall, that the ```states``` function returns the state space for a given POMDP type, and the ```domain``` function returns an iterator for a given space. Here, the domain function returns an array of the two possible states in our problem.

In [9]:
POMDPs.states(::TigerPOMDP) = TigerStateSpace([TigerState(true), TigerState(false)])
POMDPs.domain(space::TigerStateSpace) = space.states;

Recall that in the [tutorial](http://nbviewer.ipython.org/github/sisl/POMDPs.jl/blob/master/examples/GridWorld.ipynb) on MDPs, we also defined a ```rand!``` function that sampled the space. We do not need this function when using QMDP or SARSOP, so we do not define it here. However, if you wanted to use Monte Carlo solvers solvers like POMCP or DESPOT you would need a function that can sample your spaces.

### Action Space
There are three actions in our problem. Once again, we represent the action space as an array of the actions in our problem. The ```actions``` and ```domain``` functions serve a similar purpose to the ```states``` and ```domain``` functions above.

In [10]:
type TigerActionSpace <: AbstractSpace
    actions::Vector{TigerAction} 
end
# define actions function
POMDPs.actions(::TigerPOMDP) = TigerActionSpace([TigerAction(:openl), TigerAction(:openr), TigerAction(:listen)]); # default
POMDPs.actions(::TigerPOMDP, ::TigerState, acts::TigerActionSpace) = acts; # convenience (actions do not change in different states)
# define domain function
POMDPs.domain(space::TigerActionSpace) = space.actions;

### Observation Space
The observation space looks similar to the state space. Recall that the state represents the truth about our system, while the observation is potentially untuthful information recieves about the state. In the tiger POMDP, our observation could give us a false representation of our state. 

In [11]:
type TigerObservationSpace <: AbstractSpace
    obs::Vector{TigerObservation} 
end
# function returning observation space
POMDPs.observations(::TigerPOMDP) = TigerObservationSpace([TigerObservation(true), TigerObservation(false)]);
POMDPs.observations(::TigerPOMDP, s::TigerState, obs::TigerObservationSpace) = obs;
# function returning an iterator over that space
POMDPs.domain(space::TigerObservationSpace) = space.obs;

Now that we've defined the POMDP spaces, let's move on to defining the model functions.

## Transition and Observation Distributions
Before defining the model functions, we first need to create a distributions data-type. In general, our distributions should support sampling and have a ```pdf``` method. Once again, since we will not be using any sampling based solvers, we don't have to worry about implementing any sampling functions, so we only need to write a function that gives us the pdf.

Since the transition and observation distributions have identical form, we could just use a single type to serve the needs of both. This will not be the case in general, and we will define two seperate types (although similar looking) for clarity. 

In [12]:
# transition distribution type
type TigerTransitionDistribution <: AbstractDistribution
    probs::Vector{Float64} 
end
# transition distribution initializer
POMDPs.create_transition_distribution(::TigerPOMDP) = TigerTransitionDistribution([0.5, 0.5])

# observation distribution type
type TigerObservationDistribution <: AbstractDistribution
    probs::Vector{Float64} 
end
# observation distribution initializer
POMDPs.create_observation_distribution(::TigerPOMDP) = TigerObservationDistribution([0.5, 0.5]);

We now define the ```pdf``` function. For a discrete problem, this function returns the probability mass of a given element (state or observation in our case).

In [13]:
# transition pdf
function POMDPs.pdf(d::TigerTransitionDistribution, s::TigerState)
    s.tigerleft ? (return d.probs[1]) : (return d.probs[2]) 
end;
# obsevation pdf
function POMDPs.pdf(d::TigerObservationDistribution, o::TigerObservation)
    o.obsleft ? (return d.probs[1]) : (return d.probs[2])
end;

This is all we have to do for our distribution functions. Remeber that the tiger POMDP is a fairly simple problem, and you will usually need more expressive representations for your distirubtions. 

## Transition Model
Here we define the transition model for the tiger POMDP. The model itself is fairly simple. Our state is represented by the location of the tiger (left or right). The location of the tiger doesn't change when the agent listens. However, after the agent opens the door, it reaches a terminal state. That is the agent either escapes or gets eaten. To simplify our formulation, we simply reset the location of the tiger randomly. We could model this problem with a terminal state (i.e. one in which the agent no longer receives reward) as well. 

In [14]:
# the transition mode
function POMDPs.transition(pomdp::TigerPOMDP, s::TigerState, a::TigerAction, d::TigerTransitionDistribution=create_transition_distribution(pomdp))
    probs = d.probs
    # if open a door reset the tiger probs
    if a.act == :openl || a.act == :openr
        fill!(probs, 0.5)
    # if tiger is on the left, distribution = 1.0 over the first state
    elseif s.tigerleft
        probs[1] = 1.0
        probs[2] = 0.0
    # otherwise distribution = 1.0 over the second state
    else
        probs[1] = 0.0
        probs[2] = 1.0
    end
    d
end;

## Reward Model
The reward model caputres the immediate objectives of the agent. It recieves a large negative reward for opening the door with the tiger behind it (-100), gets a positive reward for opening the other door (+10), and a small penalty for listening (-1).

In [15]:
function POMDPs.reward(pomdp::TigerPOMDP, s::TigerState, a::TigerAction)
    r = 0.0
    # small penalty for listening
    if a.act == :listen
        r += pomdp.r_listen
    end
    if a.act == :openl
        # find tiger behind left door
        if s.tigerleft
            r += pomdp.r_findtiger
        # escape through left door
        else
            r += pomdp.r_escapetiger
        end
    end
    if a.act == :openr
        # escape through the right door
        if s.tigerleft
            r += pomdp.r_escapetiger
            # find tiger behind the right door
        else
            r += pomdp.r_findtiger
        end
    end
    return r
end;

## Observation Model
The observation model captures the uncertaintiy in the agent's lsitening ability. When we listen, we receive a noisy measurement of the tiger's location. 

In [16]:
function POMDPs.observation(pomdp::TigerPOMDP, s::TigerState, a::TigerAction, d::TigerObservationDistribution=create_observation_distribution(pomdp))
    probs = d.probs
    p = pomdp.p_listen_correctly # probability of listening correctly
    if a.act == :listen
        # if tiger is behind left door
        if s.tigerleft
            probs[1] = p # correct prob
            probs[2] = (1.0-p) # wring prob
        # if tiger is behind right door
        else
            probs[1] = (1.0-p) # wrong prob
            probs[2] = p # correct prob
        end
    # if don't listen uniform
    else
        fill!(probs, 0.5)
    end
    d
end;

## Miscallenous Functions

Let's define the ```discount``` function and the functions that return the size of our spaces.

In [17]:
POMDPs.discount(pomdp::TigerPOMDP) = pomdp.discount_factor
POMDPs.n_states(::TigerPOMDP) = 2
POMDPs.n_actions(::TigerPOMDP) = 3
POMDPs.n_observations(::TigerPOMDP) = 2;

## Beliefs
If you are somewhat familiar with Julia defining all of the above may have been relaitvely simple. However, all POMDPs must be represented with a belief. Implementing beliefs and their updaters can be tricky. Luckily, we provide you with some nice tools to work with beliefs. Note that if you just want to use SASROP to solve for the alpha-vectors, and use your own belief updating scheme you do not need to implement the functions below.

We will use the [POMDPToolbox](https://github.com/sisl/POMDPToolbox.jl) module which implements a discrete belief type and a belief udpater for it. 

In [ ]:
# we will use the POMDPToolbox module
using POMDPToolbox

# define a create b

In [18]:
using SARSOP
pomdp = TigerPOMDP()
policy = POMDPPolicy("tiger.policy")
pomdpfile = POMDPFile(pomdp, "tiger.pomdpx")
solver = SARSOPSolver()
solve(solver, pomdpfile, policy)


Loading the model ...
  input file   : tiger.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0       0       0        -20        92.8206    112.821     3        1        
 0       2       51       -6.2981    63.1396    69.4377     7        16       
 0       4       103      0.149651   52.2764    52.1268     9        21       
 0       6       151      6.19248    42.0546    35.8621     9        21       
 0       8       200      10.3563    35.232     24.8757     12       21       
 0       11      250      14.0433    29.5471    15.5037     6        21       
 0.01    14      300      16.545     25.0926    8.54759     10       21       
 0.01    17      350      18.2281    21.8163    3.5882      14       21   

POMDPAlphas(2x5 Array{Float64,2}:
 -81.5975   3.01448  24.6954    28.4025  19.3711
  28.4025  24.6954    3.01452  -81.5975  19.3711,[0,2,2,1,2])

In [22]:
alphas(policy)[:,1]

2-element Array{Float64,1}:
 -81.5975
  28.4025